# RetrievalQA Chain
Chat with PDF Files
___

Installing required libraries

In [1]:
#!pip install openai langchain pypdf chromadb docarray

In [2]:
import pandas as pd

I saved my OPEN AI KEY as a csv file

In [3]:
k = pd.read_csv("keys.csv")

In [4]:
import os 
import openai
os.environ['OPENAI_API_KEY'] = k["key"][0] ## <- REPLACE WITH YOUR OWN OPEN AI KEY
openai.api_key  = os.environ['OPENAI_API_KEY']

In [5]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'sagemaker-studiolab-notebooks/GenAIDemos/ChromaPDF/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

PDF Document is not read, the vectors of the different text chunks are retrieved from the database

In [6]:
print(vectordb._collection.count())

27


In [10]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [29]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain

In [30]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, 
just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

Notice the "chain" is not using memory, as a result, follow up questions could produce incorrect answers or results

In [40]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [41]:
question = "How can I fetch all the rows from a Table?"

In [42]:
result = qa_chain({"query": question})

In [43]:
print(result["result"])

To fetch all the rows from a table, you can use the sqlFetch() function in R. This function reads a table from an ODBC database into a data frame, allowing you to access all the rows and columns of the table.


In [44]:
result["source_documents"][0].metadata

{'page': 9, 'source': 'Reading from SQL Databases.pdf'}

In [45]:
print(qa_chain({"query": "How many R Datatypes are supported?"})["result"])

There are six R datatypes supported: integer, factor, numeric, character, date, and POSIXct.
